# Fetch Rewards Coding Exercise - Analytics Engineer

## 1. Review unstructured JSON data and diagram a new structured relational data model

In [83]:
#Standard Library imports

import numpy as np
import pandas as pd
import shutil                          #for operations on files
import gzip                            #to deal with different file extensions
import os
import json
import sqlalchemy
import sqlite3
from ast import literal_eval           #to evaluate strings
from datetime import datetime
import psycopg2
from sqlalchemy import create_engine

In [7]:
#Converting the unstructured JSON files to Structured data 
#User file in zipped format 
with gzip.open('users.json.gz', 'rb') as file_inpt:               #opening the file as read only
          with open('users.json.gz'.replace('.gz',''), 'wb') as file_outp:   #opening the file for writing
            shutil.copyfileobj(file_inpt, file_outp)
            

In [84]:
#Creating Dataframes for each dataset
receipts_df = pd.read_json('receipts.json',lines=True)
users_df = pd.read_json('users.json',lines=True)
brands_df = pd.read_json('brands.json',lines=True)

## Preprocessing on Receipts dataset

In [85]:
receipts_df

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,{'$oid': '603cc0630a720fde100003e6'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614594147000},{'$date': 1614594147000},NaN,{'$date': 1614594148000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"[{'barcode': 'B076FJ92M4', 'description': 'mue...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [86]:
receipts_new = receipts_df.copy()     #making a copy of receipts dataset

In [87]:
receipts_new.isna().sum()             #checking for null values

_id                          0
bonusPointsEarned          575
bonusPointsEarnedReason    575
createDate                   0
dateScanned                  0
finishedDate               551
modifyDate                   0
pointsAwardedDate          582
pointsEarned               510
purchaseDate               448
purchasedItemCount         484
rewardsReceiptItemList     440
rewardsReceiptStatus         0
totalSpent                 435
userId                       0
dtype: int64

In [88]:
#The field rewardsReceiptItemList in the receipt table contains dictionaries so 
#we will use the explode function to split the dictionaries and get multiple columns. 

receipts_df = receipts_df.explode('rewardsReceiptItemList')
receipts_df.reset_index(inplace=True)


In [89]:
receipts_df

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"{'barcode': '028400642255', 'description': 'DO...",FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"{'needsFetchReview': False, 'partnerItemId': '...",REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"{'barcode': '4011', 'description': 'ITEM NOT F...",FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
7377,1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,NaN,NaN,NaN,SUBMITTED,NaN,5fc961c3b8cfca11a077dd33
7378,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"{'barcode': 'B076FJ92M4', 'description': 'muel...",REJECTED,34.96,5fc961c3b8cfca11a077dd33
7379,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,{'$date': 1597622400000},2.0,"{'barcode': 'B07BRRLSVC', 'description': 'thin...",REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [90]:
receipts_df = receipts_df.fillna({'rewardsReceiptItemList':'{}'})      #filling missing values 
receipts_df['rewardsReceiptItemList'] = receipts_df['rewardsReceiptItemList'].apply(lambda i:str(i))  #Converting values to string
receipts_df['rewardsReceiptItemList'] = receipts_df['rewardsReceiptItemList'].apply(literal_eval)


In [91]:
receipts_normalize = pd.json_normalize(receipts_df['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList_')    



In [92]:
#merging this with the new receipts dataframe
receipts_new = pd.merge(receipts_df, receipts_normalize, left_index=True, right_index=True, how='outer')



In [93]:
receipts_new.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList_itemNumber,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,rewardsReceiptItemList_pointsEarned,rewardsReceiptItemList_targetPrice,rewardsReceiptItemList_competitiveProduct,rewardsReceiptItemList_originalFinalPrice,rewardsReceiptItemList_originalMetaBriteItemPrice,rewardsReceiptItemList_deleted,rewardsReceiptItemList_priceAfterCoupon,rewardsReceiptItemList_metabriteCampaignId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
receipts_new['_id'] = receipts_new['_id'].apply(lambda i: i['$oid'])   #extracting the values 

In [95]:
receipts_new.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList_itemNumber,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,rewardsReceiptItemList_pointsEarned,rewardsReceiptItemList_targetPrice,rewardsReceiptItemList_competitiveProduct,rewardsReceiptItemList_originalFinalPrice,rewardsReceiptItemList_originalMetaBriteItemPrice,rewardsReceiptItemList_deleted,rewardsReceiptItemList_priceAfterCoupon,rewardsReceiptItemList_metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
#Converting the date from UTC to datetime format

def date_converter(x):
    try:
        return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
    except TypeError:
        return(None)

In [97]:
date_conv_cols = ['createDate','dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for ele in date_conv_cols:     #loop to convert the columns to datetime format
    receipts_new[ele] = receipts_new[ele].apply(lambda i: date_converter(i))

In [98]:
receipts_new.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList_itemNumber,rewardsReceiptItemList_originalMetaBriteQuantityPurchased,rewardsReceiptItemList_pointsEarned,rewardsReceiptItemList_targetPrice,rewardsReceiptItemList_competitiveProduct,rewardsReceiptItemList_originalFinalPrice,rewardsReceiptItemList_originalMetaBriteItemPrice,rewardsReceiptItemList_deleted,rewardsReceiptItemList_priceAfterCoupon,rewardsReceiptItemList_metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Preprocessing on Users Dataset

In [99]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [100]:
users_df.isna().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [101]:
users_df['_id'] = users_df['_id'].apply(lambda i: i['$oid'])      #Extracting values from dictionary
users_df['createdDate'] = users_df['createdDate'].apply(lambda i: date_converter(i))     #Converting to datetime format
users_df['lastLogin'] = users_df['lastLogin'].apply(lambda i: date_converter(i))

In [102]:
users_df.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


## Preprocessing on Brands

In [103]:
brands_df.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [104]:
brands_df.isna().sum()

_id               0
barcode           0
category        155
categoryCode    650
cpg               0
name              0
topBrand        612
brandCode       234
dtype: int64

In [105]:
#removing the key value and extracting values
brands_df['_id'] = brands_df['_id'].apply(lambda i: i['$oid'])  


In [106]:
#Exploding the cpg fields

brands_normalize = pd.json_normalize(brands_df['cpg'])
brands_normalize = brands_normalize.add_prefix('cpg.')

In [107]:
brands_normalize

,cpg.$ref,cpg.$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,53e10d6368abd3c7065097cc
1164,Cogs,5332fa12e4b03c9a25efd1e7
1165,Cogs,5332f5f6e4b03c9a25efd0b4


In [108]:
#Merging the old data with the new one
brands_new = pd.merge(brands_df, brands_normalize, left_index=True, right_index=True, how='outer')



In [109]:
brands_new.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


## 2. Write a query that directly answers a predetermined question from a business stakeholder

### Connecting to SQLite

In [39]:
!pip install ipython-sql

In [40]:
!pip install SQLAlchemy==1.4.44

     |████████████████████████████████| 1.6 MB 1.1 MB/s eta 0:00:01
     |████████████████████████████████| 203 kB 11.1 MB/s eta 0:00:01
  Attempting uninstall: greenlet
    Found existing installation: greenlet 0.4.17
    Uninstalling greenlet-0.4.17:
      Successfully uninstalled greenlet-0.4.17
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 1.3.20
    Uninstalling SQLAlchemy-1.3.20:
      Successfully uninstalled SQLAlchemy-1.3.20


In [42]:
!pip install psycopg2-binary 

     |████████████████████████████████| 2.2 MB 3.3 MB/s eta 0:00:01


In [110]:
sqliteConnection = sqlite3.connect('fetch.db')     #created a database named fetch in sqlite and made a connection

In [111]:
cursor = sqliteConnection.cursor()
print("Database created and Successfully Connected to SQLite")

Database created and Successfully Connected to SQLite


In [112]:
sqlite_version = "select sqlite_version();"
cursor.execute(sqlite_version)
ver = cursor.fetchall()
print("SQLite Database Version is: ", ver)

SQLite Database Version is:  [('3.33.0',)]


In [252]:
#Pushing all the data files in the database
receipts_new.drop(columns=['rewardsReceiptItemList'],axis=1).to_sql('receipts',con=sqliteConnection)


In [253]:
users_df.drop_duplicates(subset=['_id']).to_sql('users',con=sqliteConnection)

In [254]:
brands_new.drop_duplicates(subset=['brandCode']).drop(columns=['cpg'],axis=1).to_sql('brands',con=sqliteConnection)


### What are the top 5 brands by receipts scanned for most recent month?

In [113]:
#First I checked the recent month then used this query to calculate top 5 brands for that month

receipt_sql = pd.read_sql('select br.name as Brand_name, count(*) from receipts rp left join brands br on br.brandCode = rp.rewardsReceiptItemList_brandCode where dateScanned between "2021-03-01" and "2021-03-31" group by br.name order by count(*) desc limit 5', sqliteConnection)
receipt_sql


,Brand_name,count(*)
0,None,43


### How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?

In [114]:
receipt_sql = pd.read_sql('select br.name as Brand_name, count(*) from receipts rp left join brands br on br.brandCode = rp.rewardsReceiptItemList_brandCode where dateScanned between "2021-02-01" and "2021-02-28" group by br.name order by count(*) desc limit 5', sqliteConnection)
receipt_sql


,Brand_name,count(*)
0,None,470
1,Viva,1


### When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [115]:
pd.read_sql('select rewardsReceiptStatus as Status, avg(totalSpent) as Average_spend from receipts where rewardsReceiptStatus in ("REJECTED", "FINISHED") group by rewardsReceiptStatus', sqliteConnection)

#we can see for this query that average spend from receipts with finished 
#i.e accepted status is greater than that of rejected



,Status,Average_spend
0,FINISHED,1244.372934
1,REJECTED,19.544970


### When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?

In [116]:
pd.read_sql('select rewardsReceiptStatus as Status, sum(purchasedItemCount) as Total_items from receipts where rewardsReceiptStatus in ("REJECTED", "FINISHED") group by rewardsReceiptStatus', sqliteConnection)

# we can see from this query that total items purchased from receipts with accepted status are greater


,Status,Total_items
0,FINISHED,1364998.0
1,REJECTED,740.0


## 3. Evaluate Data Quality Issues in the Data Provided

#### a. Checking if _id (primary key) in the receipt dataset is unique 

In [117]:

receipt_id = pd.read_sql('select _id, count(*) from receipts group by _id having count(*)>1', sqliteConnection)
receipt_id

,_id,count(*)
0,5f9c74f70a7214ad07000037,11
1,5f9c74f90a7214ad07000038,5
2,5fa5ad370a720f05ef000089,11
3,5fa5b0ca0a720f05ef0000bf,5
4,5fa8d5730a7214adc30001c3,11
...,...,...
297,603ce7100a7217c72c000405,2
298,603d30e60a7217c72c00043f,2
299,603d40250a720fde10000459,2
300,603d59e70a7217c72c00045f,2


#### We can see from the above query that the attribute _id i.e primary key is not unique in the receipt table. This is a data quality issue as primary key should always be unique.

#### b. Comparing the count of brand codes in the receipt and brand table

In [118]:
rec=receipts_new.dropna(subset=['rewardsReceiptItemList_brandCode'])
len(rec['rewardsReceiptItemList_brandCode'].unique())

227

In [119]:
brand_uni=brands_new.dropna(subset=['brandCode'])
len(brand_uni['brandCode'].unique())


897

#### We can see from the above query that Brand table as a lot more unique brand codes as compared to recept table. This can also lead data quality issue.

#### c. Checking in the brands table 

In [120]:

brand_check = pd.read_sql('select name, brandCode from brands where name=brandcode', sqliteConnection)
brand_check

,name,brandCode
0,MAGNUM Ice Cream,MAGNUM Ice Cream
1,THE RIGHT TO SHOWER,THE RIGHT TO SHOWER
2,CARESS,CARESS
3,SNYDERS OF HANOVER,SNYDERS OF HANOVER
4,POPSICLE,POPSICLE
...,...,...
65,FDS,FDS
66,Q-TIPS,Q-TIPS
67,BRUMMEL AND BROWN,BRUMMEL AND BROWN
68,TONI&GUY,TONI&GUY


#### We can see from the above query that for some of the records in the brand table, attribute brand name and brandcode are same. This is also a data quality issue.